In [49]:
# -*- coding: UTF-8 -*-
from math import log
import operator

In [50]:
####导入数据集
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],         #数据集
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [2, 0, 0, 0, 'no']]
    labels = ['年龄', '有工作', '有自己的房子', '信贷情况']        #分类属性
    return dataSet, labels                #返回数据集和分类属性

In [51]:
####计算熵
def calcShannonEnt(dataSet):
    numEntires = len(dataSet)                       
    labelCounts = {}                                
    #保存每个标签(Label)出现次数的字典
    for featVec in dataSet:                           
        currentLabel = featVec[-1]                    
        #提取标签(Label)信息
        if currentLabel not in labelCounts.keys():  
            #如果标签(Label)没有放入统计次数的字典,添加进去
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1                
    shannonEnt = 0.0                                
    for key in labelCounts:                         
        #计算香农熵
        prob = float(labelCounts[key]) / numEntires    
        shannonEnt -= prob * log(prob, 2)           
    return shannonEnt                                

In [52]:
####划分数据
def splitDataSet(dataSet, axis, value):       
    retDataSet = []                                        
    #创建返回的数据集列表
    for featVec in dataSet:                             
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]                
            #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])     
            retDataSet.append(reducedFeatVec)
    return retDataSet                                   

In [53]:
####选择最佳特征
def chooseBestFeatureToSplit(dataSet):
    baseEntropy = calcShannonEnt(dataSet)  
    #计算数据集的香农熵
    
    numFeatures = len(dataSet[0]) - 1                                
    bestInfoGain = 0.0                                  
    #信息增益
    bestFeature = -1                                    
    #最优特征的索引值
    for i in range(numFeatures):                     
        featList = [example[i] for example in dataSet]
        #获取dataSet的第i个所有特征
        uniqueVals = set(featList)                         
        #创建set集合{},元素不可重复
        newEntropy = 0.0                                  
        #经验条件熵
        for value in uniqueVals:                        
            subDataSet = splitDataSet(dataSet, i, value)         
            #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet))           
            #计算子集的概率
            newEntropy += prob * calcShannonEnt(subDataSet)     
            #根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy                    
        #信息增益
        print("第%d个特征的增益为%.3f" % (i, infoGain))           
        if (infoGain > bestInfoGain):                            
            bestInfoGain = infoGain                          
            bestFeature = i                                     
            #记录信息增益最大的特征的索引值
    return bestFeature                                            

In [54]:
if __name__ == '__main__':
    dataSet, features = createDataSet()
    print("最优特征索引值:", chooseBestFeatureToSplit(dataSet))

第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
最优特征索引值: 2


In [55]:
#####当无法简单地返回唯一的类标签时，返回种类最多的类别
def majorityCnt(classList):
    classCount = {}
    for vote in classList:                                        
        #统计classList中每个元素出现的次数
        classCount[vote] = classCount.get(vote,0) + 1
        
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)        
    #根据字典的值降序排序
    return sortedClassCount[0][0]                                

In [56]:
####构建决策树
def createTree(dataSet, labels, featLabels):
    classList = [example[-1] for example in dataSet]            #取分类标签(是否放贷:yes or no)
    if classList.count(classList[0]) == len(classList):            #如果类别完全相同则停止继续划分
        return classList[0]
    if len(dataSet[0]) == 1:                                    #遍历完所有特征时返回出现次数最多的类标签
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)                #选择最优特征
    bestFeatLabel = labels[bestFeat]                            #最优特征的标签
    featLabels.append(bestFeatLabel)
    myTree = {bestFeatLabel:{}}                                    #根据最优特征的标签生成树
    del(labels[bestFeat])                                        #删除已经使用特征标签
    featValues = [example[bestFeat] for example in dataSet]        #得到训练集中所有最优特征的属性值
    uniqueVals = set(featValues)                                #去掉重复的属性值
    for value in uniqueVals:                                    #遍历特征，创建决策树。                       
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), labels, featLabels)
    return myTree

In [57]:
if __name__ == '__main__':
    dataSet, labels = createDataSet()
    featLabels = []
    myTree = createTree(dataSet, labels, featLabels)
    print(myTree)

第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
第0个特征的增益为0.252
第1个特征的增益为0.918
第2个特征的增益为0.474
{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


In [58]:
def classify(inputTree, featLabels, testVec):
    firstStr = next(iter(inputTree))        #获取决策树结点
    secondDict = inputTree[firstStr]         #下一个字典
    featIndex = featLabels.index(firstStr)
    #每循环一次，featIndex都会增加1
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featLabels, testVec)
            else: classLabel = secondDict[key]
    return classLabel

In [59]:
if __name__ == '__main__':
    dataSet, labels = createDataSet()
    featLabels = []
    myTree = createTree(dataSet, labels, featLabels)
    testVec = [0,1]                                        #测试数据
    result = classify(myTree, featLabels, testVec)
    if result == 'yes':
        print('放贷')
    if result == 'no':
        print('不放贷')

第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
第0个特征的增益为0.252
第1个特征的增益为0.918
第2个特征的增益为0.474
放贷
